En esta parte continuaremos con la implementación eficiente de nuestro *modelo del lenguaje* basado en trigramas.

En el artículo anterior definimos que era un *modelo del lenguaje* la teoría necesaria para construir un modelo del lenguaje basado en n-gramas y tratamos de hacer una primera implementación utilizando una matriz como estructura de datos, lo cuál no se pudo realizar debido a la gran cantidad de memoria que sería requerida por este tipo de implementación. Esta vez utilizaremos diccionarios como estructuras de datos buscando una manera más eficiente de implementar nuestro modelo.

Comenzaremos repasando el código que hemos escrito hasta ahora para cargar nuestro corpus.

In [1]:
import re
import numpy as np

text = ''

for i in range(1, 202):
    text += (open('CORPUS/{}.txt'.format(i)).read().split('Contenido:')[-1])
    
def clean_text(text):
    text = re.sub(r'\n', '. ', text)
    text = re.sub(r'[{}@_*<>()\\#%+=\[\]ôà|è$–/—‘’«°º»”“&…∑⁄]', '', text)
    text = re.sub('a0', '', text)
    text = re.sub('\'92t', '\'t', text)
    text = re.sub('\'92s', '\'s', text)
    text = re.sub('\'92m', '\'m', text)
    text = re.sub('\'92ll', '\'ll', text)
    text = re.sub('\'91', '', text)
    text = re.sub('\'92', '', text)
    text = re.sub('\'93', '', text)
    text = re.sub('\'94', '', text)
    text = re.sub('\uf0b7', '', text)
    text = re.sub('\uf0e0', '', text)
    text = re.sub('\u200e', '', text)
    text = re.sub('\ufeff', '', text)
    text = re.sub('\u200b', '', text)
    text = re.sub('\t', '', text)
    text = re.sub('\r', '', text)
    text = re.sub('\.', '. ', text)
    text = re.sub('\!', '! ', text)
    text = re.sub('\?', '? ', text)
    text = re.sub(' +', ' ', text)
    
    return text

text = clean_text(text)

text = text.replace('.', ' _ _ ')

vocab = sorted(set(re.findall('[A-Za-z_áéíóúüñ]+', text)))

word2id = { word:i for i, word in enumerate(vocab) }

id2word = np.array(vocab)

#### Extrayendo trigramas y construyendo el modelo

Para la construcción de nuestro modelo primeramente procederemos a los trigramas el corpus para lo que se define la siguiente función:

In [2]:
def get_trigrams(text):
    # Lista para almacenar los trigramas
    trigrams = []
    
    # Segmentar el texto por palabras
    words = re.findall('[A-Za-z_áéíóúüñ]+', text)
    
    # Recorrer la lista de palabras
    for i in range(len(words)-2):
        # Extraer trigramas de la lista de palabras
        w1, w2, w3 = words[i:i+3]
        
        # Convertir palabras a su id correspondiente
        w1 = word2id[w1]
        w2 = word2id[w2]
        w3 = word2id[w3]
        
        # Agregar trigrama a la lista
        trigrams.append((w1, w2, w3))
    
    return trigrams

In [3]:
trigrams = get_trigrams(text)

In [4]:
len(trigrams)

74792

In [5]:
trigrams[:10]

[(2889, 2889, 1516),
 (2889, 1516, 8656),
 (1516, 8656, 6332),
 (8656, 6332, 11700),
 (6332, 11700, 11035),
 (11700, 11035, 5259),
 (11035, 5259, 4928),
 (5259, 4928, 9212),
 (4928, 9212, 8037),
 (9212, 8037, 7781)]

Ahora tenemos una lista con todos los trigramas extraídos de nuestro corpus, el hecho de utilizar el id de las palabras en lugar del texto nos permite ahorrar memoria además de facilitarnos la indexación como veremos a continuación.

#### Construcción del modelo

In [6]:
# Inicializamos el modelo (es un diccionario)
model = {}

In [7]:
# Contamos la frecuencia de co-ocurrencia
for i in range(len(trigrams)):
    w1, w2, w3 = trigrams[i]
    
    # El control de excepciones se encarga de manejar los distintos casos 
    # en que un trigrama aún no ha sido registrado.
    try:
        model[w1, w2][w3] += 1
    except: # Aqui se asume que w3 lanza la excepcion
        try:
            model[w1, w2][w3] = 1
        except: # Aqui se asume que el par (w1, w2) lanza la excepcion
            model[w1, w2] = {w3:1}

Las instrucciones anteriores lo que hacen es recorrer nuestra lista de trigramas e ir extrayendo cada una de las palabras que lo conforman. El diccionario que utilizamos como estructura de datos almacenará como llave las dos primeras palabras del trigrama y como valor un nuevo diccionario que tendrá como llave la tercera palabra del trigrama y como valor el número de veces que se repite el trigrama.

La primera instrucción `model[w1, w2][w3] += 1` asume que ya el trigrama completo ha sido registrado en cuyo caso solo es necesario aumentar el contador, en caso contrario se lanza una excepción y se procede a ejecutar la siguiente instrucción: `model[w1, w2][w3] = 1`, esta instrucción asume que se ha registrado ya una llave `(w1,w2)` para el primer diccionario, en este caso se procede a registrar la llave `w3` para el segundo diccionario y se inicializa su valor 1 indicando que es primera vez que se registra el trigrama. En caso de que no se hallan registrado ninguna de las llaves para ninguno de los dos diccionarios se lanzará nuevamente una excepción y se procede a registrar ambas llaves y a inicializar su valor igualmente con 1 utilizando la instrucción: `model[w1, w2] = {w3:1}`

In [8]:
# Ahora transformamos el conteo en probabilidades
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

Con esta instrucción lo que hacemos es obtener las probabilidades de ocurrencia de cada trigrama. Recorando la hipotesis markoviana $P(w_n|w_1^{n-1})$ se puede estimar como $P(w_n|w_{n-2}^{n-1})$.

Nuestro objetivo con la construcción de este modelo es poder decir dadas dos palabras cuál es la probabilidad de ocurrencia de una tercera. Probabilidad que será utlizada por nuestro corrector para ordenar las sugerencias generadas.

Y así finalmente tenemos construido nuestro modelo del lenguaje estadístico basado en trigramas.

Ahora por ejemplo determinemos cual es la probabilidad del siguiente trigrama extraído del corpus:

    definición del bien

In [9]:
sentence = "definición del bien"

w1, w2, w3 = sentence.split()

model[word2id[w1], word2id[w2]][word2id[w3]]

0.5

¿Que significa esto?, esto significa que dadas las palabras "definición" y "del" en ese orden la probabilidad de que la tercera palabra sea "bien" es de 0.5 la cuál es una probabilidad alta y para este caso quiere decir que en nuestro corpus solamente existen dos palabras que vienen despues de la cadena "definición del".

In [10]:
w1, w2 = model[word2id[w1], word2id[w2]].keys()

In [11]:
id2word[w1]

'bien'

In [12]:
id2word[w2]

'año'

Debemos señalar que los datos del modelo creado o sea los trigramas registrados y sus probabilidades de ocurrencia dependen del corpus que utilizemos, todos sabemos que después de la cadena "definición del" pueden ocurrir muchísimas palabras más pero en este caso solamente tenemos dos que han sido extraídas de nuestro corpus.

Con esto podemos decir que nuestro modelo del lenguaje esta listo, en el próximo artículo trabajaremos en la siguiente tarea requerida para la implementación de nuestro corrector estadístico: **generación de sugerencias por transformaciones de caracteres en base a inserciones, eliminaciones, sustituciones y transposición**.